# This is a Jupyter Notebook

In [1]:
# Import scikit-plot
import scikitplot as sp

sp.__version__

'0.5.0.dev0+git.20250427.9e954f7'

In [ ]:
import os

import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns